In [1]:
from keras.models import load_model
import os
import cv2
import numpy as np
import pyttsx3
import threading

In [2]:
try:
    model = load_model('emotion_model-099.keras')
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading model:", e)
    exit()

engine=pyttsx3.init()

emoji_path = {
    0: 'C:/Users/HP/Downloads/Emoji/Angry.png',
    1: 'C:/Users/HP/Downloads/Emoji/Disgusted.png',
    2: 'C:/Users/HP/Downloads/Emoji/Fear.png',
    3: 'C:/Users/HP/Downloads/Emoji/Happy.png',
    4: 'C:/Users/HP/Downloads/Emoji/Neutral.png',
    5: 'C:/Users/HP/Downloads/Emoji/Sad.png',
    6: 'C:/Users/HP/Downloads/Emoji/Surprised.png'
}
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

cap = cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier(r'C:\Users\HP\Downloads\haarcascade_frontalface_default (1).xml')


Model loaded successfully.


In [3]:
last_spoken=None

def preprocess_face(face_img):

    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (100, 100))
    resized = resized.astype('float32') / 255.0
    resized = np.reshape(resized, (1, 100, 100, 1))
    return resized


In [4]:
def overlay_emoji_on_frame(frame, emoji_img, x, y, w, h):

    emoji_resized = cv2.resize(emoji_img, (w, h))

    if emoji_resized.shape[2] == 4:
        emoji_rgb = emoji_resized[:, :, :3]  
        alpha_mask = emoji_resized[:, :, 3]  

        alpha_mask = alpha_mask / 255.0


        for c in range(0, 3): 
            frame[y:y + h, x:x + w, c] = (alpha_mask * emoji_rgb[:, :, c] + (1 - alpha_mask) * frame[y:y + h, x:x + w, c])
    else:
        frame[y:y + h, x:x + w] = cv2.addWeighted(frame[y:y + h, x:x + w], 0.5, emoji_resized, 0.5, 0)

    return frame

In [5]:
frame_count = 0
predict_interval = 5
current_emotion_label='Neutral'
def predict_emotion_async(face):
    global current_emotion_label
    processed_face = preprocess_face(face)
    emotion_prediction = model.predict(processed_face)
    max_index = int(np.argmax(emotion_prediction))
    current_emotion_label = emotion_labels[max_index]

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture video feed.")
        break
    frame = cv2.resize(frame, (320, 240)) 
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]
        
        if frame_count % predict_interval == 0:
            threading.Thread(target=predict_emotion_async, args=(face,)).start()
        
        cv2.putText(frame, current_emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        emoji_img_path = emoji_path.get(emotion_labels.index(current_emotion_label), None)
        if emoji_img_path and os.path.exists(emoji_img_path):
            emoji_img = cv2.imread(emoji_img_path, -1)
            overlay_emoji_on_frame(frame, emoji_img, x, y, w, h)
            
    frame_count += 1

    cv2.imshow('Emoji Generator from Facial Expression', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/stepte
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 809ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0